In [1]:
import os.path
import re

from common.consts import PATTERN_FILE_FORMAT__MSI, PATTERN_TYPE__BROADCAST, PATTERN_TYPE__BEAMFORMING_ELEMENT, PATTERN_TYPE__BEAMSWITCHING_SERVICE
from common.beamforming_antenna_generator import BeamformingAntennaGenerator
from common.re_filter import ReFilter
from common.pattern_name_param_extractor import PatternNameParamExtractor
from common.pattern_payload_param_extractor import PatternPayloadParamExtractor
from common.pattern_name_param_selector import PatternNameParamSelector

In [3]:
def scenario_v_port_selector(pattern_name: str) -> str | None:
    if '_PDSCH' not in pattern_name:
        return None
    if '4TRX' in pattern_name:
        return '.*4TRx.*'
    if '8TRX' in pattern_name:
        return '.*8TRx.*'
    return None


generator = BeamformingAntennaGenerator({
    # General parameters
    'src_folder': 'D:\\Cell Planning\\Antenna model processing\\Library\\mMIMO\\mMIMO Antennas\\Huawei\\HAAU5323 26G',
    'pattern_file_format': PATTERN_FILE_FORMAT__MSI,
    'version': '7.4',
    'filename': 'HAAU5323.pafx',
    'name': 'HAAU5323',
    'type': 'Cellular',
    'comment': 'mmWave HAAU5323 (26 GHz) Antenna 32.5@8T8R, 29.5@4T4R',
    'manufacturer': 'Huawei',
    'cost': 0,
    'cost_unit': 'USD',
    'length_cm': 58.5,
    'width_cm': 30,
    'depth_cm': 11,
    'weight_kg': 16,
    'wind_load_factor': 275,
    'supp_elec_tilt': True,
    'supp_elec_azimuth': False,
    'supp_elec_beamwidth': False,
    'cont_adj_elec_tilt': False,

    # Source file filter
    'src_file_re_filter': ReFilter(
        allow=['.*\.(msi|MSI)$'],
        deny=['.*CSI.*'],
    ),

    # Parameter extractors
    'pattern_name_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='(?P<cg>.+)\..{3}$',
    ),
    'scenario_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='.*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$',
        post_capture_proc=lambda r: r.replace('_SSB_', ' '),
    ),
    'v_port_name_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='.*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$',
        post_capture_proc=lambda r: r.replace('_SSB_', ' '),
    ),
    'pattern_type_extractor': PatternNameParamExtractor(
        path_part='basename',
        pre_capture_proc=lambda r: r.lower(),
        post_capture_proc=lambda r: PATTERN_TYPE__BROADCAST if 'ssb' in r else PATTERN_TYPE__BEAMSWITCHING_SERVICE,
    ),
    'center_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(payload.header['FREQUENCY'])
    ),
    'min_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(payload.header['FREQUENCY']) - 100
    ),
    'max_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(payload.header['FREQUENCY']) + 100
    ),
    'electrical_tilt_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='.*\_(?P<cg>-?\d+)T\_.*',
        post_capture_proc=lambda r: int(r),
    ),
    'polarization_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'Plus45',
    ),
    'polarization_type_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: None,
    ),
    'v_port_number_of_ports_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 1,
    ),
    'horiz_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 32,
    ),
    'horiz_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 0.6,
    ),
    'vert_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 32,
    ),
    'vert_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 0.6,
    ),
    'beamswitching_service_name_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'PDSCH' if 'PDSCH' in r else 'None',
    ),
    'beamswitching_horiz_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.horiz_pap_pattern.get_boresight_deg(),
    ),
    'beamswitching_vert_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.vert_pap_pattern.get_boresight_deg(),
    ),

    # Parameter selectors
    'scenario_selector': PatternNameParamSelector(
        select_re=scenario_v_port_selector
    ),
    'v_port_name_selector': PatternNameParamSelector(
        select_re=scenario_v_port_selector
    ),
})

[NOTICE] Could not execute regular expression: .*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$, pattern: HAAU5323_26000_X_CO_P45_00T_4TRX_PDSCH.MSI
[NOTICE] Could not execute regular expression: .*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$, pattern: HAAU5323_26000_X_CO_P45_00T_4TRX_PDSCH.MSI
[NOTICE] Could not execute regular expression: .*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$, pattern: HAAU5323_26000_X_CO_P45_00T_8TRX_PDSCH.MSI
[NOTICE] Could not execute regular expression: .*(?P<cg>\d+TRx\_SSB\_Pattern\d+)\..{3}$, pattern: HAAU5323_26000_X_CO_P45_00T_8TRX_PDSCH.MSI


In [4]:
extracted_tags = generator.list_extracted_tags(detailed=False)


Extracted tags list

[scenario]:
  1.  4TRx Pattern0...........2 items
  2.  4TRx Pattern1...........2 items
  3.  4TRx Pattern2...........2 items
  4.  8TRx Pattern0...........2 items
  5.  8TRx Pattern1...........2 items
  6.  8TRx Pattern2...........2 items

[v_port_name]:
  1.  4TRx Pattern0...........2 items
  2.  4TRx Pattern1...........2 items
  3.  4TRx Pattern2...........2 items
  4.  8TRx Pattern0...........2 items
  5.  8TRx Pattern1...........2 items
  6.  8TRx Pattern2...........2 items

[pattern_type]:
  1.  beamswitching_service....2 items
  2.  broadcast................6 items

[freq_band]:
  1.  25900-26100.............8 items

[electrical_tilt]:
  1.  0.......................8 items

[polarization]:
  1.  Plus45..................8 items

[polarization_type]:
  1.  None....................8 items

[v_port_number_of_ports]:
  1.  1.......................8 items

[horiz_number_of_elements]:
  1.  32......................8 items

[horiz_sep_dist_cm]:
  1.  0.6...........

In [5]:
generator.generate(os.path.abspath('.\output'))


Scenario > Virutal port > Virutal band > Pattern assignments
{
  "4TRx Pattern0": {
    "uid": "2",
    "name": "4TRx Pattern0",
    "horiz_number_of_elements": 32,
    "horiz_sep_dist_cm": 0.6,
    "vert_number_of_elements": 32,
    "vert_sep_dist_cm": 0.6,
    "v_ports": {
      "4TRx Pattern0": {
        "uid": "3",
        "name": "4TRx Pattern0",
        "number_of_ports": 1,
        "polarization": "Plus45",
        "polarization_type": null,
        "v_bands": {
          "25900-26100": {
            "min_freq": 25900,
            "max_freq": 26100,
            "supp_elec_tilt": true,
            "supp_elec_azimuth": false,
            "supp_elec_beamwidth": false,
            "cont_adj_elec_tilt": false,
            "broadcast_patterns": [
              "HAAU5323_26000_X_CO_P45_00T_4TRx_SSB_Pattern0"
            ],
            "electrical_controller_name": "Controller 1",
            "use_elec_params_for_bs_service_patterns": true,
            "beamforming_element_patterns": [